Coronavirus Coverage by US State
================================

A quick investigation into media coverage of the coronavirus pandemic by state in the US.
By Rahul Bhargava

## Setup

In [167]:
# Install the requirements
import sys
!{sys.executable} -m pip install -r requirements.txt

You should consider upgrading via the '/Users/rahulb/.pyenv/versions/3.7.3/envs/jupyterlab/bin/python -m pip install --upgrade pip' command.


In [168]:
from IPython.display import JSON
import os, mediacloud.api
import csv
import datetime as dt
from bokeh.io import export_svgs
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral3
from bokeh.layouts import gridplot
from bokeh.models import NumeralTickFormatter
from dotenv import load_dotenv
import json
load_dotenv()  # load config from .env file
output_notebook()

Loading BokehJS ...

In [169]:
mc = mediacloud.api.MediaCloud(os.getenv('MC_API_KEY'))
mediacloud.__version__
COVID_QUERY = 'coronavirus OR covid OR "covid 19" OR "covid-19" OR covid19 OR pandemic'
DATE_RANGE = mc.dates_as_query_clause(dt.date(2020,2,1), dt.date(2020,9,1))
US_TOP_2018_COLLECTION = 186572516
DATA_FILE = os.path.join('data', 'state-media-data.json')

## Assess High Level national coverage

In [170]:
national_covid_attention = mc.storyCount("({}) AND tags_id_media:{}".format(COVID_QUERY, US_TOP_2018_COLLECTION), DATE_RANGE, split=True)
national_attention = mc.storyCount("tags_id_media:{}".format(US_TOP_2018_COLLECTION), DATE_RANGE, split=True)
national_data = []
for day in national_attention['counts']:
    date = day['date']
    total_stories = day['count']
    covid_day = [d for d in national_covid_attention['counts'] if d['date']==date]
    covid_stories = covid_day[0]['count'] if len(covid_day) == 1 else 0
    national_data.append({'date': date, 'total': total_stories, 'covid': covid_stories})
national_df = pd.DataFrame(national_data)
national_df['date'] = pd.to_datetime(national_df['date'], format='%Y-%m-%d')
national_df = national_df.sort_values(by='date')
national_df['pct'] = national_df['covid'] / national_df['total']
national_df['rolling_pct'] = national_df.rolling(window=3)['pct'].mean()
national_df.to_csv(os.path.join('data','national-attention.csv'))
national_df.head()

,date,total,covid,pct,rolling_pct
0,2020-02-01,2360,201,0.085169,NaN
1,2020-02-02,2372,204,0.086003,NaN
2,2020-02-03,3323,373,0.112248,0.094474
3,2020-02-04,3688,422,0.114425,0.104226
4,2020-02-05,3736,408,0.109208,0.111960


In [171]:
# chart overall national attention
p = figure(plot_width=1400, plot_height=600, x_axis_type='datetime', title="Coronavirus National Coverage", y_range=(0, 1))
p.line(x='date', y='rolling_pct', line_width=1, source=ColumnDataSource(national_df), line_color='teal')
p.yaxis.formatter = NumeralTickFormatter(format='0 %')
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#p.axis.visible = False
p.outline_line_color = None
p.toolbar.logo = None
p.toolbar_location = None
p.output_backend = "svg"
export_svgs(p, filename="images/attention - top online.svg")
show(p)

## Grab Coronavirus attention by state from Media Cloud

### 1. Generate Static List of US Media by State (run just once)

In [172]:
# load all US state tags (static file coped from MediaCloud-Web-tools repository)
geo_tag_hierarchy = json.load(open(os.path.join('data','mc-geo-adm1.json')))['byCountry']
us = [country for country in geo_tag_hierarchy if country['country']['alpha3'] == 'USA'][0]
# filter out the "national" and combined "state & local" collections
us_state_collections = [s for s in us['collections'] if not s['label'].startswith('United States')]
# add in a handy state name
for s in us_state_collections:
    s['name'] = s['label'].split(',')[0]
us_state_collections[0]

{'show_on_stories': False,
 'label': 'Alabama, United States - State & Local',
 'tag_sets_id': 15765102,
 'tag_set_description': 'Tags in this set indicate that the media source covers a certain geographic area',
 'tags_id': 38381313,
 'tag_set_label': 'Geographic Collections',
 'is_static': False,
 'tag_set_name': 'geographic_collection',
 'show_on_media': True,
 'tag': 'geo_US-AL',
 'description': 'Media is largely about Alabama, United States - State & Local',
 'name': 'Alabama'}

In [173]:
# now get the media in each collection
def all_media_list(**kwargs):
    # page through a list of media list results (copied from Media Cloud API Tutorial notebooks)
    last_media_id = None
    more_results = True
    matching_media = []
    while more_results:
        media_page = mc.mediaList(**kwargs, last_media_id=last_media_id)
        #print("  got a page of {} matching media".format(len(media_page)))
        if len(media_page) == 0:
            more_results = False
        else:
            matching_media += media_page
            last_media_id = media_page[-1]['media_id']
    return matching_media

pr_collection = mc.tag(34412297)
pr_collection['name'] = "Puerto Rico"
pr_collection['tag'] = "geo_US-PR"
us_state_collections.append(pr_collection)

for s in us_state_collections:
    state_media = all_media_list(tags_id=s['tags_id'], rows=500)
    s['media'] = state_media
    s['media_count'] = len(state_media)
    s['avg_stories_per_day'] = round(sum([m['num_stories_90'] for m in state_media]))
    print("{} - {} media sources ({} stories/day)".format(s['name'], len(s['media']), s['avg_stories_per_day']))

Alabama - 146 media sources (921 stories/day)
Alaska - 80 media sources (681 stories/day)
Arizona - 181 media sources (3104 stories/day)
Arkansas - 141 media sources (1342 stories/day)
California - 1453 media sources (13915 stories/day)
Colorado - 233 media sources (1776 stories/day)
Connecticut - 126 media sources (682 stories/day)
Delaware - 29 media sources (281 stories/day)
District of Columbia - 49 media sources (1079 stories/day)
Florida - 335 media sources (5286 stories/day)
Georgia - 231 media sources (1891 stories/day)
Hawaii - 46 media sources (524 stories/day)
Idaho - 74 media sources (215 stories/day)
Illinois - 335 media sources (4962 stories/day)
Indiana - 188 media sources (1604 stories/day)
Iowa - 173 media sources (1137 stories/day)
Kansas - 168 media sources (1658 stories/day)
Kentucky - 141 media sources (775 stories/day)
Louisiana - 134 media sources (903 stories/day)
Maine - 75 media sources (637 stories/day)
Maryland - 105 media sources (1496 stories/day)
Massachu

In [174]:
# cache state collections locally in a file so we don't hit the server over and over
with open(DATA_FILE, 'w') as outfile:
    json.dump(us_state_collections, outfile)

### 2. Add in State Coronavirus Attention Data (run just once)

In [175]:
# load the static file of media sources by state, which now includes media lists for each state
us_state_collections = json.load(open(DATA_FILE))
len(us_state_collections)

52

In [176]:
for s in us_state_collections:
    # add in the total number of stories published from sources in this state
    s['total_stories'] = mc.storyCount("tags_id_media:{}".format(s['tags_id']), DATE_RANGE, split=True)['counts']
    # add in the total number of stories ABOUT COVID published from sources in this state
    s['covid_stories'] = mc.storyCount("tags_id_media:{} AND {}".format(s['tags_id'], COVID_QUERY), DATE_RANGE, split=True)['counts']

In [177]:
# generate CSVs of attention data for each state
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    state_csv_file = os.path.join('data', 'state-attention', "{}-stories.csv".format(state_abbr))
    s['state_csv_file'] = state_csv_file
    data = []
    for day in s['total_stories']:
        date = day['date']
        total_stories = day['count']
        covid_day = [d for d in s['covid_stories'] if d['date']==date] # if there were no stories, day won't have entry in covid results
        covid_stories = covid_day[0]['count'] if len(covid_day) == 1 else 0
        data.append({'state': s['name'],
                     'date': date,
                     'total': total_stories,
                     'covid': covid_stories
                    })
    with open(state_csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['state', 'date', 'total', 'covid'], extrasaction='ignore')
        writer.writeheader()
        for d in data:
            writer.writerow(d)
# and add the CSV path to the main file
with open(DATA_FILE, 'w') as outfile:
    json.dump(us_state_collections, outfile)

### 3. Chart Attention Data

In [178]:
order = ['AK', None, None, None, None, None, None, None, None, 'VT', 'NH', 'ME',
        None, None, None, None, None, None, None, None, 'NY', 'CT', 'MA', None,
        'WA', 'MT', 'ND', 'SD', 'MN', 'WI', 'MI', None, 'PA', 'NJ', 'RI', None,
        'OR', 'ID', 'WY', 'NE', 'IA', 'IL', 'IN', 'OH', 'VA', 'DC', 'DE', None,
        'NV', 'UT', 'CO', 'KS', 'MO', 'TN', 'KY', 'WV', 'NC', 'MD', None, None,
        'CA', 'AZ', 'NM', 'OK', 'AR', 'MS', 'AL', 'GA', 'SC', None, None, None,
        None, None, None, 'TX', 'LA', None, None, None, 'FL', None, None, None,
        'HI', None, None, None, None, None, None, None, None, None, None, 'PR',]
def chunks(lst, n):
    #https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [179]:
# load the static file of media sources by state, which now also has attention data
us_state_collections = json.load(open(DATA_FILE))
total_stories = sum([d['count'] for s in us_state_collections for d in s['total_stories']])
covid_stories = sum([d['count'] for s in us_state_collections for d in s['covid_stories']])
"{:.2%} of stories are about corona ({:,} / {:,})".format((covid_stories/total_stories), covid_stories, total_stories)

'31.70% of stories are about corona (3,975,110 / 12,541,639)'

In [180]:
# make a dataset for each state
media_state_df = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    state_df = pd.read_csv(os.path.join('data','state-attention','{}-stories.csv'.format(state_abbr)))
    state_df['date'] = pd.to_datetime(state_df['date'], format='%Y-%m-%d')
    state_df = state_df.sort_values(by='date')
    state_df['pct'] = state_df['covid'] / state_df['total']
    state_df['rolling_pct'] = state_df.rolling(window=3)['pct'].mean()
    state_df.dropna()
    media_state_df[state_abbr] = state_df
media_state_df['MA'].head()

,state,date,total,covid,pct,rolling_pct
0,Massachusetts,2020-02-01,865,96,0.110983,NaN
1,Massachusetts,2020-02-02,1121,67,0.059768,NaN
2,Massachusetts,2020-02-03,1793,138,0.076966,0.082572
3,Massachusetts,2020-02-04,1963,161,0.082017,0.072917
4,Massachusetts,2020-02-05,2024,74,0.036561,0.065182


In [181]:
# make a combined chart of all states
# make a chart for each state
media_plots_by_abbr = {}
p = figure(plot_width=1400, plot_height=600, x_axis_type='datetime', title="Coronavirus Coverage by State", y_range=(0, 1))
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    df = media_state_df[state_abbr]
    source = ColumnDataSource(df)
    p.line(x='date', y='rolling_pct', line_width=1, source=source, line_color='teal')
p.yaxis.formatter = NumeralTickFormatter(format='0 %')
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
#p.axis.visible = False
p.outline_line_color = None
p.toolbar.logo = None
p.toolbar_location = None
show(p)

In [182]:
# make a chart for each state
media_plots_by_abbr = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    df = media_state_df[state_abbr]
    # now make a chart for the state
    source = ColumnDataSource(df)
    p = figure(plot_width=140, plot_height=140, x_axis_type='datetime', title=s['name'], title_location="below", y_range=(0, 1))
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.outline_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    p.line(x='date', y='rolling_pct', line_width=1, source=source, line_color='teal')
    media_plots_by_abbr[state_abbr] = p
    #show(p)
# now let's replicate R's geofacet library by hand
geo_aligned_media_plots = []
for abbr in order:
    plot = media_plots_by_abbr[abbr] if abbr is not None else None
    geo_aligned_media_plots.append(plot)
rows = list(chunks(geo_aligned_media_plots,12))
grid = gridplot(rows)
show(grid)

## Grab Coronavirus incidence/death data from John Hopkins

In [183]:
# download the latest data from their repository on GitHub
import wget, zipfile
JH_GH_REPO_URL = "https://github.com/CSSEGISandData/COVID-19/archive/master.zip"
dest_path = os.path.join('tmp/jh-repo.zip')
wget.download(JH_GH_REPO_URL, dest_path)
with zipfile.ZipFile(dest_path, 'r') as zip_ref:
    zip_ref.extractall('tmp')

In [184]:
# aggregate the covid data by state so we can chart it
import pathlib, dateparser
state_data_path = os.path.join('tmp', 'COVID-19-master', 'csse_covid_19_data', 'csse_covid_19_daily_reports_us') # this folder has one CSV per day and one row per state
state_csv_files = [os.path.join(state_data_path, f) for f in os.listdir(state_data_path) if f.endswith('.csv')]
print("Found {} daily files".format(len(state_csv_files)))
state_name2abbr = {t['name']:t['tag'][-2:] for t in us_state_collections}
state_data = []
for data_file in state_csv_files:
    state_covid_df = pd.read_csv(data_file)
    for index, row in state_covid_df.iterrows():
        if row['Province_State'] in state_name2abbr:
            item = {
                'state': state_name2abbr[row['Province_State']],
                'date': dateparser.parse(pathlib.PurePath(data_file).name[:-3]),
                'cases': row['Confirmed'],
                'deaths': row['Deaths'],
                'incidence': row['Incident_Rate'],
            }
            state_data.append(item)
# write combined CSV of state-level data by day
with open(os.path.join('data','covid-by-state.csv'), 'w') as fd:
    csv_writer = csv.DictWriter(fd, ['state', 'date', 'cases', 'deaths', 'incidence'])
    csv_writer.writeheader()
    csv_writer.writerows(state_data) 
print("Wrote {} daily state records".format(len(state_data)))

Found 152 daily files
Wrote 7904 daily state records


In [185]:
data_cols = ['cases', 'deaths', 'incidence']

# cleanup and average a little
jh_df = pd.read_csv(os.path.join('data','covid-by-state.csv'))
jh_df['date'] = pd.to_datetime(jh_df['date'], format='%Y-%m-%d')
valid_range = (jh_df['date'] > '02-01-2020') & (jh_df['date'] <= '09-01-2020')
jh_df = jh_df.loc[valid_range]
# apply a window to smooth out weekends and show more of a trend chart (ie. not as spikey)
jh_df['cases'] = pd.to_numeric(jh_df['cases'])
jh_df['deaths'] = pd.to_numeric(jh_df['deaths'])
jh_df['incidence'] = pd.to_numeric(jh_df['incidence'])
jh_df = jh_df.dropna()
#jh_df.head()

# make a dataset for each state
covid_state_df = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    state_df = jh_df[jh_df['state'] == state_abbr]
    state_df = state_df.sort_values(by='date')
    # the columns are are cumulatve, so do a shift and subtract to get new-per-day
    state_df[data_cols] -= state_df[data_cols].shift(1).fillna(0)
    state_df = state_df[1:] # dump the first day because after the shift/subtract it isn't valid
    state_df['rolling_incidence'] = state_df.rolling(window=3)['incidence'].mean()
    state_df.dropna()
    covid_state_df[state_abbr] = state_df
covid_state_df['MA'].head()

,state,date,cases,deaths,incidence,rolling_incidence
2673,MA,2020-04-13,1392.0,88.0,20.280394,NaN
1373,MA,2020-04-14,1297.0,0.0,18.896315,NaN
1321,MA,2020-04-15,1754.0,264.0,25.554462,21.577057
4285,MA,2020-04-16,2263.0,0.0,32.970209,25.806996
4337,MA,2020-04-17,2221.0,137.0,32.358301,30.294324


In [186]:
# now build each data set into a plot by state
covid_plots_by_abbr = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    state_df = covid_state_df[state_abbr]
    # now make a chart for the state
    source = ColumnDataSource(state_df)
    p = figure(plot_width=140, plot_height=140, x_axis_type='datetime', title=s['name'], title_location="below", y_range=(0, 100))
    p.line(x='date', y='rolling_incidence', line_width=1, source=source, line_color='purple')
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.outline_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    covid_plots_by_abbr[state_abbr] = p
    #show(p)
# now let's replicate R's geofacet library by hand
covid_geo_aligned_plots = []
for abbr in order:
    plot = covid_plots_by_abbr[abbr] if abbr is not None else None
    covid_geo_aligned_plots.append(plot)
rows = list(chunks(covid_geo_aligned_plots,12))
grid = gridplot(rows)
show(grid)

## Compare incidence rate and media coverage for each state

In [187]:
# pull together the two datesets, by state/day
merged_state_df = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    media_df = media_state_df[state_abbr]
    covid_df = covid_state_df[state_abbr]
    merged_state_df[state_abbr] = pd.merge(media_df, covid_df, left_on='date', right_on='date')
merged_state_df['MA'].head()

,state_x,date,total,covid,pct,rolling_pct,state_y,cases,deaths,incidence,rolling_incidence
0,Massachusetts,2020-04-13,2688,1730,0.643601,0.665307,MA,1392.0,88.0,20.280394,NaN
1,Massachusetts,2020-04-14,2784,1626,0.584052,0.632226,MA,1297.0,0.0,18.896315,NaN
2,Massachusetts,2020-04-15,3243,1734,0.534690,0.587448,MA,1754.0,264.0,25.554462,21.577057
3,Massachusetts,2020-04-16,3288,1848,0.562044,0.560262,MA,2263.0,0.0,32.970209,25.806996
4,Massachusetts,2020-04-17,2575,1352,0.525049,0.540594,MA,2221.0,137.0,32.358301,30.294324


In [188]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d
merged_state_plots = {}
for s in us_state_collections:
    state_abbr = s['tag'].split('-')[1]
    df = merged_state_df[state_abbr]
    p = figure(plot_width=140, plot_height=140, x_axis_type='datetime', title=s['name'], title_location="below")
    # add media coverage pct
    p.line(df['date'], df['rolling_pct'], line_width=1, color="teal")
    p.y_range = Range1d(0,1)
    # add covid incidence rate (per 100,000)
    p.extra_y_ranges = { 'incidence_range': Range1d(start=0, end=100)}
    p.add_layout(LinearAxis(y_range_name='incidence_range'), "right")
    p.line(df['date'], df['rolling_incidence'], line_width=1, y_range_name='incidence_range', color='purple')
    # clean up for display
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.outline_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    merged_state_plots[state_abbr] = p
#show(p)
# now let's replicate R's geofacet library by hand
merged_geo_aligned_plots = []
for abbr in order:
    plot = merged_state_plots[abbr] if abbr is not None else None
    merged_geo_aligned_plots.append(plot)
rows = list(chunks(merged_geo_aligned_plots,12))
grid = gridplot(rows)
show(grid)

In [189]:
# dig into certain states
spotlight_state_collections = [s for s in us_state_collections if s['tag'].split('-')[1] in ['MA', 'AZ']]
plots = []
for s in spotlight_state_collections:
    state_abbr = s['tag'].split('-')[1]
    df = merged_state_df[state_abbr]
    p = figure(plot_width=500, plot_height=500, x_axis_type='datetime', title=s['name'], title_location="below")
    # add media coverage pct
    p.line(df['date'], df['rolling_pct'], line_width=2, color="teal") #, legend_label="stories")
    p.y_range = Range1d(0,1)
    # add covid incidence rate (per 100,000)
    p.extra_y_ranges = { 'incidence_range': Range1d(start=0, end=100)}
    p.add_layout(LinearAxis(y_range_name='incidence_range'), "right")
    p.line(df['date'], df['rolling_incidence'], line_width=2, y_range_name='incidence_range', color='purple') #legend_label="incidence per 100,000")
    # clean up for display
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.outline_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    #p.legend.location = "top_right"
    #p.legend.click_policy="hide"
    plots.append(p)
grid = gridplot([plots])
show(grid)